In [83]:
import fuzzywuzzy.process
import importlib
import json
import os
import pandas as pd
import PyPDF3
import get_pdf_fields

In [3]:
# Determine the fields corresponding to each entry in a PDF.
importlib.reload(get_pdf_fields)
pdf_fp = './Annual_Check-in_Questionnaire_Draft_07.09.19.pdf'
items_fp = './items.csv'
gpf = get_pdf_fields.GetPdfFields(pdf_fp, items_fp)

In [ ]:
gpf.set_field()
gpf.current_item

In [70]:
gpf.write(True, True)

In [126]:
# Manual analysis and comparison.
def diff(pdf_fp):
    """
    Parameters: fields: list
                    The list of PDF file paths.
    Returns:    w: list
                    The list of key, value pairs for the PDF fields with
                    different values.
    """
    
    w = []
    # Define the two PDF file paths.
    fields = [PyPDF3.PdfFileReader(x).getFields() for x in pdf_fp]
    for key, val in fields[0].items():
        val_copy = fields[1][key]
        if '/V' in val.keys():
            if '/V' in val_copy.keys():
                if val['/V'] != val_copy['/V']:
                    w += [(key, val['/V'])]
            else:
                w += [(key, val['/V'])]
    return w
    
pdf_fp = ['./Annual_Check-in_Questionnaire_Draft_07.09.19.pdf']
pdf_fp += ['Annual_Check-in_Questionnaire_Draft_07.09.19 (copy).pdf']
d = diff(pdf_fp)
d

[]

In [116]:
# Get the proper item number and risk level for each item.
json_fp = './Annual_Check-in_Questionnaire_Draft_07.09.19.json'
xlsx_fp = './Risk_Profile_of_ACQ_07.03.19.xlsx'
# Import the JSON file.
with open(json_fp, 'r') as f:
    json_data = json.load(f)
# Import the Excel file.
xlsx_data = pd.read_excel(xlsx_fp, sheet_name='Question Characteristics')
# Add modified group and question columns for ease in matching.
def fstr(x):
    if isinstance(x, str):
        return x.lower().strip().replace(',', '')
    else:
        return x
xlsx_data['topic_match'] = xlsx_data['Topic'].apply(fstr)
xlsx_data['q_match'] = xlsx_data['Question'].apply(fstr)
# Group key map to account for differences in the XLSX and JSON files.
group_map = {}
group_map['General Information'] = ''
val = 'Net Payment Reconciliation Amount (NPRA)'
group_map['Net Payment Reconciliation Report (NPRA)'] = val
key = 'Waivers (Fraud & Abuse Waivers and Medicare Payment Policy Waivers)'
val = 'Waivers (Fraud & Abuse and Medicare Payment Policy)'
group_map[key] = val
# Loop through each element in json data and find the associated ID and risk.
check = []
for key, val in json_data.items():
    # We account for differences in the JSON and XLSX files.
    if val['group'] in group_map.keys():
        group = fstr(group_map[val['group']])
    else:
        group = fstr(val['group'])
    if not group == '':
        # The question info.
        xlsx_group = xlsx_data[xlsx_data['topic_match'] == group].copy()
        text = fstr(val['text'])
        xlsx_q = xlsx_group[xlsx_group['q_match'] == text].copy()
        if not len(xlsx_q) == 1:
            # In this case we need fuzzy matching.
            match = fuzzywuzzy.process.extractOne(text, xlsx_group['q_match'])
            # Find the data associated to the match.
            xlsx_q = xlsx_group[xlsx_group['q_match'] == match[0]].copy()
        qid = xlsx_q.iloc[0]['ID']
        qrisk = xlsx_q.iloc[0]['Risk Level']
        json_data[key]['id'] = int(qid)
        json_data[key]['risk'] = int(qrisk)
with open(json_fp, 'w') as f:
    json.dump(json_data, f, indent=2)

In [115]:
json_data

{'Text3': {'num': 0,
  'local': 0,
  'text': 'BPID',
  'group': 'General Information'},
 'Text1_21': {'num': 1,
  'local': 1,
  'text': 'Organization Name',
  'group': 'General Information'},
 'ComboBox7': {'num': 2,
  'local': 1,
  'text': 'Does your organization have experience with value-based care beyond BPCI Advanced?',
  'group': 'General Information Requested',
  'id': 1,
  'risk': 4},
 'ComboBox7_12': {'num': 3,
  'local': 2,
  'text': 'Does your organization have current or past experience with CMS Innovation Center models and/or Medicare demonstrations, beyond BPCI Advanced?',
  'group': 'General Information Requested',
  'id': 2,
  'risk': 4},
 'ComboBox8': {'num': 4,
  'local': 1,
  'text': 'Has your organization, or any of its Model partners, had a legal name change, change of control or change of tax identifier in the last year?',
  'group': 'Organization Legal or Identification Changes',
  'id': 5,
  'risk': 3},
 'ComboBox9': {'num': 5,
  'local': 2,
  'text': "If select

In [97]:
match

('if yes have you reported these changes to cmmi?', 86, 5)

In [109]:
xlsx_group[xlsx_group['q_match'] == match[0]]

,Topic,Source,ID,Question,Risk Level,topic_match,q_match
5,Organization Legal or Identification Changes,Participant Response,6.0,"If yes, have you reported these changes to CMMI?",7,organization legal or identification changes,if yes have you reported these changes to cmmi?


In [104]:
match[0]

'if yes have you reported these changes to cmmi?'